In [1]:
# Import the libraries you need
import pandas as pd # Pandas is like excel, but in Python
import numpy as np # Numpy is for working with matrices, which is useful for calculating indicator values
import random # Used for random sampling when we run different combinations of parameters

In [2]:
# Read in the data
file_path = '../bar_movement/data/' # Goes to the folder where the data is held
currency_pair = 'Eur_Usd' # Currency pair we want to run the simulation for

df = pd.read_csv(file_path + 'Oanda_Eur_Usd_M5_2022-2023.csv') # Reads in the csv file you want
df.Date = pd.to_datetime(df.Date, utc=True) # Make sure the date is an object we can call methods on

In [3]:
# Show the first 5 rows
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume
0,2022-06-15 06:00:00+00:00,1.04625,1.04678,1.04614,1.04637,1.04645,1.04697,1.04635,1.04655,1.04635,1.04688,1.04624,1.04646,1218
1,2022-06-15 06:05:00+00:00,1.04637,1.04651,1.04543,1.04547,1.04654,1.04670,1.04559,1.04564,1.04646,1.04660,1.04552,1.04556,946
2,2022-06-15 06:10:00+00:00,1.04551,1.04594,1.04514,1.04541,1.04569,1.04611,1.04534,1.04558,1.04560,1.04602,1.04524,1.04550,813
3,2022-06-15 06:15:00+00:00,1.04539,1.04630,1.04519,1.04588,1.04558,1.04650,1.04538,1.04607,1.04548,1.04640,1.04529,1.04598,1152
4,2022-06-15 06:20:00+00:00,1.04586,1.04779,1.04582,1.04775,1.04605,1.04797,1.04601,1.04792,1.04596,1.04788,1.04592,1.04784,1271


In [4]:
# Show the last 5 rows
df.tail()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume
74773,2023-06-15 05:35:00+00:00,1.08162,1.08196,1.08155,1.08165,1.08175,1.08210,1.08171,1.08180,1.08168,1.08203,1.08164,1.08172,353
74774,2023-06-15 05:40:00+00:00,1.08166,1.08188,1.08161,1.08183,1.08182,1.08203,1.08177,1.08196,1.08174,1.08195,1.08169,1.08190,317
74775,2023-06-15 05:45:00+00:00,1.08182,1.08213,1.08180,1.08200,1.08197,1.08228,1.08195,1.08216,1.08190,1.08220,1.08188,1.08208,248
74776,2023-06-15 05:50:00+00:00,1.08200,1.08228,1.08191,1.08198,1.08214,1.08243,1.08206,1.08213,1.08207,1.08236,1.08198,1.08206,333
74777,2023-06-15 05:55:00+00:00,1.08199,1.08228,1.08177,1.08177,1.08213,1.08243,1.08193,1.08193,1.08206,1.08236,1.08185,1.08185,263


In [5]:
# Function for the ATR indicator
def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).mean()

In [6]:
# Add indicators to the dataframe
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

# Make sure there aren't any null values and make sure the row numbers are sequential
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [7]:
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,ema200,macd,macdsignal,atr
0,2022-06-15 07:05:00+00:00,1.04701,1.04752,1.04631,1.04745,1.04717,1.04768,1.04645,1.04762,1.04709,1.04760,1.04638,1.04754,759,1.047413,0.000057,0.000120,0.001010
1,2022-06-15 07:10:00+00:00,1.04748,1.04812,1.04748,1.04798,1.04763,1.04831,1.04763,1.04815,1.04756,1.04822,1.04756,1.04806,949,1.047459,0.000080,0.000112,0.001013
2,2022-06-15 07:15:00+00:00,1.04798,1.04838,1.04772,1.04798,1.04813,1.04858,1.04790,1.04815,1.04806,1.04848,1.04782,1.04806,953,1.047499,0.000096,0.000109,0.000983
3,2022-06-15 07:20:00+00:00,1.04798,1.04844,1.04758,1.04843,1.04816,1.04862,1.04776,1.04861,1.04807,1.04853,1.04767,1.04852,934,1.047564,0.000136,0.000114,0.000989
4,2022-06-15 07:25:00+00:00,1.04843,1.04873,1.04778,1.04862,1.04863,1.04891,1.04795,1.04877,1.04853,1.04882,1.04786,1.04870,786,1.047633,0.000176,0.000127,0.000978


In [9]:
amounts_per_day = [-0.8, -1, -1.2] if 'Jpy' in currency_pair else [-0.08, -0.1, -0.12]

def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

In [16]:
def run_simulation(pips_to_risk, risk_reward_ratio, invert, trail_stop_loss):
    reward, day_fees, n_buys, n_sells, n_wins, n_losses, longest_win_streak, longest_loss_streak, curr_win_streak, curr_loss_streak = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    trade = None
    amount_per_loss = 50
    amount_per_win = amount_per_loss * risk_reward_ratio

    for i in range(2, len(df)):
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        mid_close, ema200, macd1, macdsignal1 = df.loc[df.index[i - 1], ['Mid_Close', 'ema200', 'macd', 'macdsignal']]
        ask_open, bid_open, ask_high, ask_low, bid_high, bid_low, curr_date, mid_open, bid_close = df.loc[df.index[i], ['Ask_Open', 'Bid_Open', 'Ask_High', 'Ask_Low', 'Bid_High', 'Bid_Low', 'Date', 'Mid_Open', 'Bid_Close']]

        # Check if we should open a trade
        if trade is None:
            buy_signal = mid_close > ema200 and macd2 < macdsignal2 and macd1 > macdsignal1 and max([macd2, macdsignal2, macd1, macdsignal1]) < 0
            sell_signal = mid_close < ema200 and macd2 > macdsignal2 and macd1 < macdsignal1 and min([macd2, macdsignal2, macd1, macdsignal1]) > 0

            if invert:
                buy_signal, sell_signal = sell_signal, buy_signal

            # For buys
            if buy_signal:
                open_price = ask_open
                stop_loss = open_price - pips_to_risk
                take_profit = open_price + (risk_reward_ratio * pips_to_risk)
                n_units = get_n_units('buy', stop_loss, ask_open, bid_open, mid_open, currency_pair)

                trade = {'open_price': open_price, 'stop_loss': stop_loss, 'take_profit': take_profit, 'trade_type': 'buy', 'start_date': curr_date, 'n_units': n_units, 'prev_profit_ratio': None}

                n_buys += 1

            # For sells
            elif sell_signal:
                open_price = bid_open
                stop_loss = open_price + pips_to_risk
                take_profit = open_price - (risk_reward_ratio * pips_to_risk)
                n_units = get_n_units('sell', stop_loss, ask_open, bid_open, mid_open, currency_pair)

                trade = {'open_price': open_price, 'stop_loss': stop_loss, 'take_profit': take_profit, 'trade_type': 'sell', 'start_date': curr_date, 'n_units': n_units, 'prev_profit_ratio': None}

                n_sells += 1

        # Check if the trade would've closed out
        if trade is not None:
            if trade['trade_type'] == 'buy' and bid_low < trade['stop_loss']:
                trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units']
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date)

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > longest_win_streak:
                    longest_win_streak = curr_win_streak

                if curr_loss_streak > longest_loss_streak:
                    longest_loss_streak = curr_loss_streak

                trade = None

            elif trade['trade_type'] == 'buy' and trail_stop_loss is None and bid_high > trade['take_profit']:
                reward += amount_per_win # Equivalent to reward = reward + amount_per_loss
                day_fees += calculate_day_fees(trade['start_date'], curr_date)
                n_wins += 1

                curr_win_streak += 1
                curr_loss_streak = 0
                longest_win_streak = max(longest_win_streak, curr_win_streak)

                trade = None

            elif trade['trade_type'] == 'sell' and ask_high > trade['stop_loss']:
                trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units']
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date)

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > longest_win_streak:
                    longest_win_streak = curr_win_streak

                if curr_loss_streak > longest_loss_streak:
                    longest_loss_streak = curr_loss_streak

                trade = None

            elif trade['trade_type'] == 'sell' and trail_stop_loss is None and ask_low < trade['take_profit']:
                reward += amount_per_win
                day_fees += calculate_day_fees(trade['start_date'], curr_date)
                n_wins += 1

                curr_win_streak += 1
                curr_loss_streak = 0
                longest_win_streak = max(longest_win_streak, curr_win_streak)

                trade = None

            if trade is not None and trail_stop_loss == 'traditional' and trade['trade_type'] == 'buy' and bid_high - pips_to_risk > trade['stop_loss']:
                trade['stop_loss'] = bid_high - pips_to_risk

            if trade is not None and trail_stop_loss == 'traditional' and trade['trade_type'] == 'sell' and ask_low + pips_to_risk < trade['stop_loss']:
                trade['stop_loss'] = ask_low + pips_to_risk

    return reward, day_fees, n_buys, n_sells, n_wins, n_losses, longest_win_streak, longest_loss_streak

In [20]:
pips_to_risk_vals = [0.0010, 0.0020, 0.0030]
risk_reward_ratio_vals = [0.5, 1.0, 1.5, 2.0, 3.0]
invert_vals = [True]
trail_stop_losses = ['traditional']

all_combos = []

for pips_to_risk in pips_to_risk_vals:
    for risk_reward_ratio in risk_reward_ratio_vals:
        for invert in invert_vals:
            for trail_stop_loss in trail_stop_losses:
                all_combos.append((pips_to_risk, risk_reward_ratio, invert, trail_stop_loss))

percentage_to_try = 1.0
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

top_n_results = 5
best_rewards = []
best_reward = -np.inf
runs_finished = 0
best_pips_to_risk, best_risk_reward_ratio, best_invert_val, best_trail_stop_loss = None, None, None, None

for pips_to_risk, risk_reward_ratio, invert, trail_stop_loss in combos_to_try:
    reward, day_fees, n_buys, n_sells, n_wins, n_losses, longest_win_streak, longest_loss_streak = run_simulation(pips_to_risk, risk_reward_ratio, invert, trail_stop_loss)
    runs_finished += 1

    total_profit = reward + day_fees

    print(f'Reward: {reward}')
    print(f'Day fees: {day_fees}')   
    print(f'Reward + day fees: {total_profit}')  
    print(f'# Buys: {n_buys}')
    print(f'# Sells: {n_sells}')
    print(f'# Wins: {n_wins}')
    print(f'# Losses: {n_losses}')
    print(f'Longest win streak: {longest_win_streak}')
    print(f'Longest loss streak: {longest_loss_streak}')
    print('Remaining runs: ' + str(n_runs - runs_finished))

    # Keep track of the best top_n_results results
    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or total_profit > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(total_profit), 'pips_to_risk': pips_to_risk, 'risk_reward_ratio': risk_reward_ratio, 'invert': invert, 'trail_stop_loss': trail_stop_loss})

    # Keep track of the very best result
    if total_profit > best_reward:
        best_reward = total_profit
        best_pips_to_risk, best_risk_reward_ratio, best_invert_val, best_trail_stop_loss = pips_to_risk, risk_reward_ratio, invert, trail_stop_loss

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 15

Reward: -1348.699979999879
Day fees: -14.999999999999973
Reward + day fees: -1363.699979999879
# Buys: 291
# Sells: 321
# Wins: 204
# Losses: 407
Longest win streak: 6
Longest loss streak: 23
Remaining runs: 14
Best reward so far: -1363.699979999879

Reward: -4813.951579995475
Day fees: -9.419999999999991
Reward + day fees: -4823.371579995475
# Buys: 433
# Sells: 486
# Wins: 302
# Losses: 616
Longest win streak: 5
Longest loss streak: 14
Remaining runs: 13
Best reward so far: -1363.699979999879

Reward: -4813.951579995475
Day fees: -9.179999999999996
Reward + day fees: -4823.1315799954755
# Buys: 433
# Sells: 486
# Wins: 302
# Losses: 616
Longest win streak: 5
Longest loss streak: 14
Remaining runs: 12
Best reward so far: -1363.699979999879

Reward: -4813.951579995475
Day fees: -9.499999999999991
Reward + day fees: -4823.451579995475
# Buys: 433
# Sells: 486
# Wins: 302
# Losses: 616
Longest win streak: 5
Longest loss streak: 14
Remaining runs: 11
Best reward so far: -136

In [18]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best pips to risk: ' + str(best_pips_to_risk))
print('Best risk reward ratio: ' + str(best_risk_reward_ratio))
print('Best trailing stop loss: ' + str(best_trail_stop_loss))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 437.7284400006796
Best pips to risk: 0.003
Best risk reward ratio: 3.0
-----------------------
Top results:
{'reward': 437, 'pips_to_risk': 0.003, 'risk_reward_ratio': 3.0, 'invert': True, 'trail_stop_loss': 'traditional'}
{'reward': 437, 'pips_to_risk': 0.003, 'risk_reward_ratio': 0.5, 'invert': True, 'trail_stop_loss': 'traditional'}
{'reward': 437, 'pips_to_risk': 0.003, 'risk_reward_ratio': 1.5, 'invert': True, 'trail_stop_loss': 'traditional'}
{'reward': 230, 'pips_to_risk': 0.003, 'risk_reward_ratio': 3.0, 'invert': True, 'trail_stop_loss': None}
{'reward': 437, 'pips_to_risk': 0.003, 'risk_reward_ratio': 2.0, 'invert': True, 'trail_stop_loss': 'traditional'}
